In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
import glob
import os

df = []
for csv in tqdm(glob.glob('simdata/*/*/*.csv')):
    
    _, pref, year, meshcode = os.path.splitext(csv)[0].split('/')
    # print(pref, year, meshcode)
    if int(year) > 1985:
        continue
    
    _df = pd.read_csv(csv)[['date', 'year', 'meshcode', 'DVI']]
    _df['pref'] = pref
    df.append(_df)
    
df = pd.concat(df, axis=0)
df.head()

In [ ]:
df.groupby(['pref', 'year', 'meshcode']).max()